In [0]:
#importing necessary libraries
from pyspark.sql import SparkSession, Window
from pyspark.sql import functions as f
from pyspark.sql.types import *
from delta.tables import DeltaTable

In [0]:
#bronze data

# Path on S3 that has all our Raw files produced by the websocket listener
inputPath = "/alpaca/crypto/btc/bars/"

# Our raw data is written to S3 as a json file, Spark needs us to provide a schema before being able to read the data
jsonSchema = StructType([
  StructField("close", StringType(), True),
  StructField("exchange", StringType(), True),
  StructField("high", StringType(), True),
  StructField("low", StringType(), True),
  StructField("open", StringType(), True),
  StructField("symbol", StringType(), True),
  #StructField("year", StringType(),True),
  #StructField("date", DateType(),True),
  StructField("timestamp", LongType(), True),
  StructField("trade_count", StringType(), True),
  StructField("volume", StringType(), True),
  StructField("vwap", StringType(), True)])

# Lets create a Structured Streaming DataFrame using readStream. Note, the the crypto timestamp is in epoch time (unix)
# So it is necessary to convert this to a usable format before creating our Silve Table
streamingInputDF = (
  spark
    .readStream
    .schema(jsonSchema)
    .json(inputPath)
    .withColumn('timestamp', f.from_utc_timestamp(f.from_unixtime(f.col('timestamp')/1000000000, "yyyy-MM-dd hh:mm:ss"), tz='America/New_York'))
)

In [0]:
#converting variable created in above command into a temp view to use SQL against for visualization
streamingInputDF.createOrReplaceTempView("bronze2")

In [0]:
#silver table
streamingInputDF.writeStream \
  .format("delta") \
  .outputMode("append") \
  .option("checkpointLocation", "/alpaca/crypto/btc/silver_bars/_checkpoints/elt-from-json") \
  .start("/alpaca/crypto/btc/silver_bars") 

Out[4]: <pyspark.sql.streaming.StreamingQuery at 0x7ff15c6f5640>

In [0]:
#calculating and visualizing the sum of the volume, by hour
gold = (
  streamingInputDF
    .groupBy(
      streamingInputDF.volume,
      f.window(streamingInputDF.timestamp, "1 hour"))
    .sum()
)

display(gold)

volume,window
0.96315688,"List(2021-11-27T22:00:00.000+0000, 2021-11-27T23:00:00.000+0000)"
1.0940788,"List(2021-11-28T00:00:00.000+0000, 2021-11-28T01:00:00.000+0000)"
1.37533154,"List(2021-11-28T01:00:00.000+0000, 2021-11-28T02:00:00.000+0000)"
2.15000172,"List(2021-11-28T00:00:00.000+0000, 2021-11-28T01:00:00.000+0000)"
16.77409793,"List(2021-11-28T02:00:00.000+0000, 2021-11-28T03:00:00.000+0000)"
13.59323156,"List(2021-11-28T01:00:00.000+0000, 2021-11-28T02:00:00.000+0000)"
5.6786487,"List(2021-11-27T23:00:00.000+0000, 2021-11-28T00:00:00.000+0000)"
1.73005538,"List(2021-11-27T22:00:00.000+0000, 2021-11-27T23:00:00.000+0000)"
13.05469146,"List(2021-11-28T01:00:00.000+0000, 2021-11-28T02:00:00.000+0000)"
1.09310687,"List(2021-11-28T00:00:00.000+0000, 2021-11-28T01:00:00.000+0000)"


In [0]:
#converting variable created in above command into a temp view to use SQL against for visualization
gold.createOrReplaceTempView("gold2")

In [0]:
%sql

SELECT *

FROM gold2

volume,window
0.96315688,"List(2021-11-27T22:00:00.000+0000, 2021-11-27T23:00:00.000+0000)"
1.0940788,"List(2021-11-28T00:00:00.000+0000, 2021-11-28T01:00:00.000+0000)"
1.37533154,"List(2021-11-28T01:00:00.000+0000, 2021-11-28T02:00:00.000+0000)"
2.15000172,"List(2021-11-28T00:00:00.000+0000, 2021-11-28T01:00:00.000+0000)"
4.24736479,"List(2021-11-28T02:00:00.000+0000, 2021-11-28T03:00:00.000+0000)"
16.77409793,"List(2021-11-28T02:00:00.000+0000, 2021-11-28T03:00:00.000+0000)"
13.59323156,"List(2021-11-28T01:00:00.000+0000, 2021-11-28T02:00:00.000+0000)"
5.6786487,"List(2021-11-27T23:00:00.000+0000, 2021-11-28T00:00:00.000+0000)"
1.73005538,"List(2021-11-27T22:00:00.000+0000, 2021-11-27T23:00:00.000+0000)"
13.05469146,"List(2021-11-28T01:00:00.000+0000, 2021-11-28T02:00:00.000+0000)"


In [0]:
%sql
--count of trades by day in the bronze table
SELECT exchange,
  symbol,
  date(timestamp) AS date,
  hour(timestamp) AS hour,
  sum(volume) AS vol_per_hour

FROM bronze2

GROUP BY exchange,
  symbol,
  date(timestamp),
  hour(timestamp)

ORDER BY date(timestamp) DESC

In [0]:
#write to gold table
gold.writeStream \
  .format("delta") \
  .outputMode("complete") \
  .option("checkpointLocation", "/alpaca/crypto/btc/gold_bars/_checkpoints/hourly-sum") \
  .start("/alpaca/crypto/btc/gold_bars2")
  
display(gold, processingTime = "60 seconds") 

volume,window
0.96315688,"List(2021-11-27T22:00:00.000+0000, 2021-11-27T23:00:00.000+0000)"
1.0940788,"List(2021-11-28T00:00:00.000+0000, 2021-11-28T01:00:00.000+0000)"
1.37533154,"List(2021-11-28T01:00:00.000+0000, 2021-11-28T02:00:00.000+0000)"
2.15000172,"List(2021-11-28T00:00:00.000+0000, 2021-11-28T01:00:00.000+0000)"
4.24736479,"List(2021-11-28T02:00:00.000+0000, 2021-11-28T03:00:00.000+0000)"
16.77409793,"List(2021-11-28T02:00:00.000+0000, 2021-11-28T03:00:00.000+0000)"
13.59323156,"List(2021-11-28T01:00:00.000+0000, 2021-11-28T02:00:00.000+0000)"
5.6786487,"List(2021-11-27T23:00:00.000+0000, 2021-11-28T00:00:00.000+0000)"
1.73005538,"List(2021-11-27T22:00:00.000+0000, 2021-11-27T23:00:00.000+0000)"
13.05469146,"List(2021-11-28T01:00:00.000+0000, 2021-11-28T02:00:00.000+0000)"


In [0]:
gold.schema

Out[7]: StructType(List(StructField(volume,StringType,true),StructField(window,StructType(List(StructField(start,TimestampType,true),StructField(end,TimestampType,true))),false)))